## 数据处理

In [2]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl
from model_classifyV1_Copy1 import postprocess_excel_by_topic

In [3]:
## 研发字典
speaker_map = {
    "16186514":   "研发peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt      = "1105《欢迎来到地球》测试1群.txt"

# 设定时间范围
start_time = "2025-10-22 00:00:00"
end_time   = "2025-11-4 09:00:00"

# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



20593 lines
{"发言日期": "2025-10-22", "发言时间": "00:00:37", "玩家ID": "阿雪(1138152183)", "玩家消息": "[图片]"}


## 大模型分类

## 定义

In [4]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：对保留内容进行 5 类意图分类（输出带“意图分类”的 JSON 行）
- 结果按 sheet（体感反馈/疑惑不解和问题询问/玩家建议和灵感/情绪输出/问题反馈）写入 Excel
"""
from model_classifyV1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter, build_user_prompt_classify,build_user_prompt_classify2,
    call_ark_chat_completions,
    jsonl_to_dataframe_with_intent,
    create_intent_excel_styled, append_json_to_excel_by_cat_and_tag,load_whitelist,
    extract_clusters_from_output, update_and_save_whitelist, build_user_prompt_cluster_correct
)

## 设置参数

In [5]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md")      # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("提示词2.md")      # 模型#2 system 提示词（做分类）
PROMPT_MD_PATH03 = Path ("提示词5.md")     # 模型#3 system 提示词（做话提簇）
PROMPT_MD_PATH04 = Path ("话提簇校正提示词.md")     # 模型#3 system 提示词（做话提簇修正）

EXCEL_FILE       = Path("1022_1104.xlsx")   # 输出 Excel
BATCH_SIZE       = 100
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 开始运行

In [ ]:
from tqdm import tqdm
import time

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做分类
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03) # 做话提簇
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04) # 做话提簇

create_intent_excel_styled(EXCEL_FILE)


# 植入白名单 #
# --- 白名单初始化 ---
WHITE_LIST_PATH = Path("话题簇白名单.jsonl")
if not WHITE_LIST_PATH.exists():
    WHITE_LIST_PATH.write_text("", encoding="utf-8")  # 自动创建空文件
whitelist = load_whitelist(WHITE_LIST_PATH)  # 首批为空或加载已有



total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        # --- 模型 #2：分类 ---
        user_prompt2 = build_user_prompt_classify(output_filter)
        output_classify = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_classify:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        # --- 模型 #3：二级标签 ---
        user_prompt3 = build_user_prompt_classify2(output_classify)
        output_classify2 = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt03,
            user_prompt=user_prompt3,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        # --- 模型 #4：话提簇校正 ---
        user_prompt4 = build_user_prompt_cluster_correct(output_classify2, whitelist)
        output_classify3 = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_1_MODEL_ID,
        system_prompt=system_prompt04,  # 模型#4 提示词
        user_prompt=user_prompt4,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
        )
        try:
            new_clusters = extract_clusters_from_output(output_classify3)
            if new_clusters:
                whitelist = update_and_save_whitelist(WHITE_LIST_PATH, whitelist, new_clusters)
            else:
                tqdm.write(f"[批次 {b+1}] ⚪ 本批次无新增话题簇。")
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠️ 白名单更新出错：{e}")

        # 转 DataFrame
        df_batch = jsonl_to_dataframe_with_intent(output_classify3)
        if df_batch is None or df_batch.empty:
            tqdm.write(f"[批次 {b+1}] ⚠️ 无可写入数据（空 DataFrame）。")
            continue

        # ✅ 关键：转为 list[dict] 再写入
        records = df_batch.to_dict(orient="records")
        append_json_to_excel_by_cat_and_tag(records, EXCEL_FILE)

        batch_written = len(records)
        written_total += batch_written
        tqdm.write(f"[批次 {b+1}] ✅ 写入 {batch_written} 条。")
     
        
  
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        
        continue

    time.sleep(SLEEP_BETWEEN)
postprocess_excel_by_topic(EXCEL_FILE, gap_minutes=60, nat_policy="skip")

print("\n✅ 全部批次处理完成！")
print(f"输入总数：{total}")
print(f"写入总数：{written_total}")


✅ 创建并套样式：1022_1104.xlsx
准备处理 20593 条，共 206 批（每批 100 条）。


🔥 批处理进度:   0%|                                                                                | 0/206 [04:58<?, ?批/s]      

✅ 新增 11 条话题簇至白名单
[批次 1] ✅ 写入 32 条。


🔥 批处理进度:   0%|▎                                                                   | 1/206 [06:49<17:02:14, 299.19s/批]      

✅ 新增 4 条话题簇至白名单
[批次 2] ✅ 写入 11 条。


...

🔥 批处理进度:  13%|████████▏                                                        | 26/206 [2:56:44<24:51:44, 497.25s/批]

✅ 新增 10 条话题簇至白名单


🔥 批处理进度:  13%|████████▏                                                        | 26/206 [3:06:23<24:51:44, 497.25s/批]      

[批次 27] ✅ 写入 81 条。


🔥 批处理进度:  13%|████████▌                                                        | 27/206 [3:06:24<25:57:43, 522.14s/批]